In [1]:
import pandas as pd

df = pd.read_csv('/Users/marcelosilva/Desktop/projectOne/3/D-Variable Analysis/CleanDataset.csv')

In [ ]:
import pandas as pd
import re
import os

def parse_variable_descriptions(file_path):
    """
    Lê o arquivo de descrições e extrai nome_variável + descrição
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        print(f"✅ Arquivo carregado: {file_path}")
        
        # Dicionário para armazenar variável: descrição
        variables_dict = {}
        
        # Dividir por linhas
        lines = content.strip().split('\n')
        
        for line in lines:
            line = line.strip()
            if line:
                # Tentar diferentes padrões de separação
                # Padrão 1: variável<tab>descrição
                if '\t' in line:
                    parts = line.split('\t', 1)
                    if len(parts) == 2:
                        var_name = parts[0].strip()
                        description = parts[1].strip()
                        variables_dict[var_name] = description
                
                # Padrão 2: variável - descrição
                elif ' - ' in line:
                    parts = line.split(' - ', 1)
                    if len(parts) == 2:
                        var_name = parts[0].strip()
                        description = parts[1].strip()
                        variables_dict[var_name] = description
                
                # Padrão 3: variável espaços descrição
                else:
                    parts = line.split()
                    if len(parts) >= 2:
                        var_name = parts[0].strip()
                        description = ' '.join(parts[1:]).strip()
                        variables_dict[var_name] = description
        
        print(f"📊 Variáveis extraídas: {len(variables_dict)}")
        
        # Mostrar algumas amostras
        print(f"\n📋 Amostra das variáveis encontradas:")
        for i, (var, desc) in enumerate(list(variables_dict.items())[:5]):
            print(f"   {var} → {desc}")
        
        return variables_dict
        
    except FileNotFoundError:
        print(f"❌ Arquivo não encontrado: {file_path}")
        return None
    except Exception as e:
        print(f"❌ Erro ao carregar arquivo: {e}")
        return None

def load_clean_dataset_columns(file_path):
    """
    Carrega apenas as colunas do CleanDataset
    """
    try:
        df = pd.read_csv(file_path, nrows=0)  # Só cabeçalho
        columns = df.columns.tolist()
        
        print(f"✅ CleanDataset colunas carregadas: {len(columns)} variáveis")
        return columns
        
    except Exception as e:
        print(f"❌ Erro ao carregar CleanDataset: {e}")
        return None

def create_simple_crossref(variables_dict, dataset_columns, output_path):
    """
    Cria cruzamento simples: variável + descrição
    """
    print(f"\n🔍 CRUZAMENTO SIMPLES")
    print("=" * 30)
    
    # Identificar variáveis que estão em ambos
    available_variables = []
    
    for var_name in dataset_columns:
        if var_name in variables_dict:
            available_variables.append({
                'variable': var_name,
                'description': variables_dict[var_name]
            })
    
    print(f"✅ Variáveis com descrição no CleanDataset: {len(available_variables)}")
    
    # Criar arquivo simples
    os.makedirs(output_path, exist_ok=True)
    
    # 1. Arquivo TXT simples
    txt_path = os.path.join(output_path, "variables_for_feature_engineering.txt")
    
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write("VARIÁVEIS DISPONÍVEIS PARA FEATURE ENGINEERING\n")
        f.write("=" * 50 + "\n\n")
        
        for item in available_variables:
            f.write(f"{item['variable']}\t{item['description']}\n")
    
    # 2. Arquivo CSV simples
    csv_path = os.path.join(output_path, "variables_for_feature_engineering_simple.csv")
    
    df_simple = pd.DataFrame(available_variables)
    df_simple.to_csv(csv_path, index=False)
    
    # 3. Arquivo Markdown organizado
    md_path = os.path.join(output_path, "variables_for_feature_engineering_simple.md")
    
    with open(md_path, 'w', encoding='utf-8') as f:
        f.write("# Variables for Feature Engineering\n\n")
        f.write(f"## Available Variables ({len(available_variables)})\n\n")
        f.write("| Variable | Description |\n")
        f.write("|----------|-------------|\n")
        
        for item in available_variables:
            # Escapar caracteres especiais do markdown
            desc_clean = item['description'].replace('|', '\\|')
            f.write(f"| `{item['variable']}` | {desc_clean} |\n")
        
        f.write(f"\n## Variables Without Description\n\n")
        
        variables_without_desc = [var for var in dataset_columns if var not in variables_dict]
        
        if variables_without_desc:
            f.write("The following variables are in CleanDataset but lack descriptions:\n\n")
            for var in variables_without_desc:
                f.write(f"- `{var}`\n")
        else:
            f.write("All variables in CleanDataset have descriptions! ✅\n")
    
    print(f"\n📄 Arquivos criados:")
    print(f"   📝 TXT: {txt_path}")
    print(f"   📊 CSV: {csv_path}")
    print(f"   📋 MD: {md_path}")
    
    # Mostrar resultado na tela
    print(f"\n📋 RESULTADO DO CRUZAMENTO:")
    print("=" * 50)
    
    for item in available_variables:
        print(f"{item['variable']}\t{item['description']}")
    
    return available_variables

def main():
    """
    Processo principal simples
    """
    print("CRUZAMENTO SIMPLES: VARIÁVEL + DESCRIÇÃO")
    print("=" * 50)
    
    # Caminhos
    described_vars_path = "/Users/marcelosilva/Desktop/projectOne/1/D-Data Characteristics/variaveis_descritas_90pct.txt"
    clean_dataset_path = "/Users/marcelosilva/Desktop/projectOne/3/D-Variable Analysis/CleanDataset.csv"
    output_path = "/Users/marcelosilva/Desktop/projectOne/4"
    
    # 1. Carregar descrições
    variables_dict = parse_variable_descriptions(described_vars_path)
    
    # 2. Carregar colunas do dataset
    dataset_columns = load_clean_dataset_columns(clean_dataset_path)
    
    if variables_dict is None or dataset_columns is None:
        print("❌ Erro ao carregar arquivos!")
        return
    
    # 3. Fazer cruzamento simples
    result = create_simple_crossref(variables_dict, dataset_columns, output_path)
    
    print(f"\n🎉 CONCLUÍDO!")
    print(f"📊 {len(result)} variáveis disponíveis para feature engineering")
    print(f"📁 Arquivos salvos em: {output_path}")

# EXECUTAR
if __name__ == "__main__":
    main()